In [ ]:
# Assignment 1: Training a neural network with Iris-dataset

In [8]:
# Importing the libraries needed
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras as K
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten
from keras.utils import to_categorical

In [ ]:
# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
data = pd.read_csv(url, names=columns)
print(data.head())
print(data.shape)

# Shuffle the data
data = data.sample(frac=1).reset_index(drop=True)

# One-hot encode
data['class'] = pd.Categorical(data['species']).codes

# Split the data into input and output features
x = pd.DataFrame(data.drop('class', axis=1))
y = to_categorical(data['class'])

print(x.head())
print(y[:5])


   sepal_length  sepal_width  petal_length  petal_width      species
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa
(150, 5)
   sepal_length  sepal_width  petal_length  petal_width          species
0           6.3          3.4           5.6          2.4   Iris-virginica
1           5.3          3.7           1.5          0.2      Iris-setosa
2           6.7          3.3           5.7          2.5   Iris-virginica
3           5.0          2.0           3.5          1.0  Iris-versicolor
4           4.8          3.1           1.6          0.2      Iris-setosa
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
